# Import dataset

In [8]:

import json
from pprint import pprint

import pandas as pd
from ibm_watsonx_ai import Credentials
from langchain.retrievers import EnsembleRetriever, TFIDFRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_ibm import WatsonxLLM, WatsonxEmbeddings

# Define the file path
csv_file_path = './output/menus.csv'

# Read the CSV file and assign headers explicitly
df = pd.read_csv(csv_file_path, header=None, names=["FileName", "Text"])

# Display the first few rows of the DataFrame
display(df)

# Confirm the data has been successfully loaded
print(f"DataFrame loaded with {len(df)} rows and {len(df.columns)} columns.")


,FileName,Text
0,Sapore del Dune.pdf,"Ristorante ""Sapore del Dune""\nChef Alessandra ..."
1,Universo Gastronomico di Namecc.pdf,Universo Gastronomico di Namecc\nChef Alice Qu...
2,L Equilibrio Quantico.pdf,"Ristorante ""L'Equilibrio Quantico""\nChef Aless..."
3,L Architetto dell Universo.pdf,"Ristorante ""L'Architetto dell'Universo""\nChef ..."
4,L Essenza Cosmica.pdf,"Ristorante ""L'Essenza Cosmica""\n\nChef Alessan..."
5,Stelle Astrofisiche.pdf,"Ristorante ""Stelle Astrofisiche""\nChef Alessan..."
6,L Essenza di Asgard.pdf,Ristorante: L'Essenza di Asgard\nChef Palissan...
7,Eco di Pandora.pdf,"Ristorante ""L'Eco di Pandora""\nChef Alessandra..."
8,L Eco dei Sapori.pdf,L'Eco dei Sapori\nChef Aurora Vessanti\n\nNell...
9,L Essenza delle Dune.pdf,"Ristorante ""L'Essenza delle Dune""\nChef Alessa..."


DataFrame loaded with 30 rows and 2 columns.


In [10]:
loader = DataFrameLoader(df, page_content_column="Text")
docs_data = loader.load()
docs_data[0].model_json_schema()


{'description': 'Class for storing a piece of text and associated metadata.\n\nExample:\n\n    .. code-block:: python\n\n        from langchain_core.documents import Document\n\n        document = Document(\n            page_content="Hello, world!",\n            metadata={"source": "https://example.com"}\n        )',
 'properties': {'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'default': None,
   'title': 'Id'},
  'metadata': {'title': 'Metadata', 'type': 'object'},
  'page_content': {'title': 'Page Content', 'type': 'string'},
  'type': {'const': 'Document',
   'default': 'Document',
   'title': 'Type',
   'type': 'string'}},
 'required': ['page_content'],
 'title': 'Document',
 'type': 'object'}

In [11]:
# Possible improvements - future hypertuning of chunk_size and chunk_overlap to improve results and try different slitters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
splits = text_splitter.split_documents(docs_data)
pprint(splits[0:6])
pprint(len(splits))


[Document(metadata={'FileName': 'Sapore del Dune.pdf'}, page_content='Ristorante "Sapore del Dune"\nChef Alessandra Quanti'),
 Document(metadata={'FileName': 'Sapore del Dune.pdf'}, page_content='Nel cuore arido di Tatooine, dove i mondi si mescolano e le stelle guidano i viaggiatori intergalattici, Chef\nAlessandra Quanti porta una rivoluzione culinaria che sfida le distanze siderali. Non è raro vedere i\ncommensali rimanere incantati osservando i suoi piatti che sembrano danzare tra le dune e le stelle, frutto\ndella sua straordinaria padronanza degli stati quantici, che le permette di esplorare e materializzare le infinite\npossibilità nascoste in ogni ingrediente rarefatto del deserto.'),
 Document(metadata={'FileName': 'Sapore del Dune.pdf'}, page_content="La sua storia ebbe inizio nei laboratori di spezie di Mos Eisley, dove la passione per la gastronomia\nmolecolare si fuse con la sua profonda comprensione dell'universo subatomico. Fu proprio durante un\nesperimento particolarme

# Watsonx

In [12]:
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key="AnzfgthfcrfRzttoXGiKZUJDMRlcB3w4uemf0PJGFFT5"
)

OVERWRITE = False


In [13]:
watsonLLM = WatsonxLLM(
    model_id="mistralai/mistral-large",  # Che conosciamo bene 😊🏆
    url=credentials['url'],
    apikey=credentials['api_key'],
    project_id="5c33debb-5a25-4bfe-8392-ede4b20884fe",
    params={
        "max_new_tokens": 1024
    }
)


In [14]:
embeddings = WatsonxEmbeddings(
    model_id="ibm/granite-embedding-107m-multilingual",
    url=credentials["url"],
    apikey=credentials["api_key"],
    project_id="5c33debb-5a25-4bfe-8392-ede4b20884fe",
)


In [16]:
if OVERWRITE:
    vectorstore = FAISS.from_documents(splits, embeddings)
    vectorstore.save_local("local_model_index")


In [18]:
vectorstore = FAISS.load_local("local_model_index", embeddings, allow_dangerous_deserialization=True)
vectorstore.index.ntotal


1638

In [19]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
retriever


VectorStoreRetriever(tags=['FAISS', 'WatsonxEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f2c23124590>, search_kwargs={'k': 5})

In [20]:
# Initialize the BM25 retriever
sparse_retriever = TFIDFRetriever.from_documents(splits)
sparse_retriever.k = 5

# Initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[sparse_retriever, retriever], weights=[0.4, 0.6])


In [21]:
# Prompt
template = """
Rispondi a domande relative a piatti e ristoranti galattici. 
Dato il contesto estratto dai documenti relativi a ristoranti di vari pianeti, ognuno con un menu di piatti e specifici ingredienti, 
rispondi alla domanda del cliente basandoti solo su queste informazioni.

Istruzioni:
1. Leggi solo il contesto fornito, che include descrizioni dei piatti, degli ingredienti e delle tecniche.
2. Ritorna esclusivamente l'elenco dei nomi dei piatti che rispondono alla domanda, senza alcuna informazione aggiuntiva.
3. Assicurati di scrivere la risposta come nel seguente formato: ["Piatto 1", "Piatto 2"].

Contesto: {context}
Domanda: {question}"
"""

prompt = ChatPromptTemplate.from_template(template)


# Post-processing
def format_docs(splits):
    return "\n\n".join(doc.page_content for doc in splits)


# Chain
rag_chain = (
        {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | watsonLLM
        | StrOutputParser()
)

# Question
question = "Quali piatti sono preparati utilizzando la tecnica della Sferificazione a Gravità Psionica Variabile?"
response_text = rag_chain.invoke(question)
print(response_text)



Assistente: ["Sinfonia Cosmica di Andromeda", "Risveglio Cosmico: Una Sinfonia di Sapori Universali", "Odissea Celestiale"]


In [22]:
pprint(response_text)


('\n'
 'Assistente: ["Sinfonia Cosmica di Andromeda", "Risveglio Cosmico: Una '
 'Sinfonia di Sapori Universali", "Odissea Celestiale"]')


In [23]:
def levenshtein_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2 + 1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]


In [24]:
with open('./data/Misc/dish_mapping.json', 'r') as f:
    json_data = json.load(f)


In [37]:
def generate_submission(json_data=json_data):
    import re, ast
    domande_test = pd.read_csv('./data/domande.csv')
    out = pd.DataFrame(columns=['row_id', 'result'])
    i = 1

    for q in domande_test['domanda']:
        response = rag_chain.invoke(q)
        match = re.search(r'\[[^\]]*\]', response)
        string_list = ast.literal_eval(match.group(0))

        string_list_proj = []

        for s1 in string_list:
            opt = float('inf')
            s1_proj = ''

            for s2 in list(json_data.keys()):
                dist = levenshtein_distance(s1, s2)
                if dist < opt:
                    opt = dist
                    s1_proj = s2

            string_list_proj += [json_data[s1_proj]]

        result = ",".join(str(s) for s in string_list_proj) if len(string_list_proj) > 0 else '1'

        out = pd.concat([out, pd.DataFrame(
            data=[[int(i), result]],
            columns=['row_id', 'result']
        )], ignore_index=True)
        i = i + 1

    return out


In [38]:
out = generate_submission()
out.to_csv(path_or_buf='submission.csv', index=False)


In [39]:
out


,row_id,result
0,1,101
1,2,"180,201,192"
2,3,52
3,4,215
4,5,1
...,...,...
95,96,"57,57,52,81,39,207,184,151"
96,97,"72,224,283"
97,98,"74,209"
98,99,250
